# Diffusion Model 


##Resources:
**Papers and Survey**</br>
https://arxiv.org/pdf/1503.03585.pdf</br>
https://arxiv.org/pdf/2006.11239.pdf</br>
https://arxiv.org/pdf/2105.05233.pdf</br>
https://arxiv.org/pdf/2102.09672.pdf</br>
https://arxiv.org/pdf/2209.00796.pdf</br>
https://arxiv.org/pdf/2206.02262.pdf</br>
**Article**</br>
https://lilianweng.github.io/posts/2021-07-11-diffusion-models/</br>
https://towardsai.net/p/l/gan-is-diffusion-all-you-need</br>
**Videos**</br>
https://www.youtube.com/watch?v=a4Yfz2FxXiY</br>
https://www.youtube.com/watch?v=cS6JQpEY9cs</br>
**Paper Explained**</br>
https://www.youtube.com/watch?v=W-O7AZNzbzQ</br>
**Math Explained**</br>
https://www.youtube.com/watch?v=HoKDTa5jHvg</br>
**Jeremy Howard**</br>
https://www.youtube.com/watch?v=_7rMfsA24Ls&feature=youtu.be</br>
https://www.youtube.com/watch?v=0_BBRNYInx8</br>
https://www.youtube.com/watch?v=mYpjmM7O-30</br>
**Codes**</br>
https://github.com/lucidrains/denoising-diffusion-pytorch</br>
https://github.com/NVlabs/stylegan3</br>

# 1 - Environment Preparation:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/sde.py .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/UNET.py .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/utils.py .
!cp -avr /content/drive/MyDrive/Colab\ Notebooks/modified\ version/diffusion_utils.py . 

'/content/drive/MyDrive/Colab Notebooks/modified version/sde.py' -> './sde.py'
'/content/drive/MyDrive/Colab Notebooks/modified version/UNET.py' -> './UNET.py'
'/content/drive/MyDrive/Colab Notebooks/modified version/utils.py' -> './utils.py'
'/content/drive/MyDrive/Colab Notebooks/modified version/diffusion_utils.py' -> './diffusion_utils.py'


In [ ]:
 pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.0 MB/s 


In [ ]:
!pip install pytorch-ignite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 264 kB 5.3 MB/s 


# 2 - Defining the model parameters 

In [14]:
import argparse
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from sde import VariancePreservingSDE, PluginReverseSDE
from utils import get_grid
from utils import LogitTransform
from UNET import UNet
from utils import logging, create
from tensorboardX import SummaryWriter
import json


_folder_name_keys = ['dataset', 'real', 'debias', 'batch_size', 'lr', 'num_iterations']



# i/o
dataset='mnist'
dataroot='/content/drive/MyDrive/Colab Notebooks/modified version/datasets'
saveroot ='/content/drive/MyDrive/Colab Notebooks/modified version/saved'
expname = 'default'
print_every= 500
sample_every=500
checkpoint_every=1000
num_steps= 1000
FID_every = 1000
num_iterations=10000
# optimization
T0=1.0
vtype= 'rademacher'
batch_size=64
test_batch_size=256
lr=0.0001
# model
real=True
debias=False


folder_tag = 'sde-flow'
folder_name = '-'.join([ k for k in _folder_name_keys])
create(saveroot, folder_tag, expname, folder_name)
folder_path = os.path.join(saveroot, folder_tag, expname, folder_name)
print_ = lambda s: logging(s, folder_path)
print_(f'folder path: {folder_path}')
writer = SummaryWriter(folder_path)

input_channels = 1
input_height = 28
dimx = input_channels * input_height ** 2

transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.MNIST(root=dataroot, train=True,
                                      download=True, transform=transform)
testset = torchvision.datasets.MNIST(root=dataroot, train=False,
                                      download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=test_batch_size,
                                          shuffle=True, num_workers=2)

model = UNet(
    input_channels=input_channels,#channels
    input_height=input_height, #image_size
    ch=32,#dim
    ch_mult=(1, 2, 2),
    num_res_blocks=2,
    attn_resolutions=(16,),
    resamp_with_conv=True,
)
T = torch.nn.Parameter(torch.FloatTensor([T0]), requires_grad=False)

inf_sde = VariancePreservingSDE(beta_min=0.1, beta_max=20.0, T=T)
sde_model = PluginReverseSDE(inf_sde, model, T, vtype=vtype, debias=debias)

cuda = torch.cuda.is_available()
if cuda:
    sde_model.cuda()

optim = torch.optim.Adam(sde_model.parameters(), lr=lr)

logit = LogitTransform(alpha=0.05)
if real:
    reverse = logit.reverse
else:
    reverse = None

2022-11-25 19:48:18.913713	folder path: /content/drive/MyDrive/Colab Notebooks/modified version/saved/sde-flow/default/dataset-real-debias-batch_size-lr-num_iterations
1
2


"for x_test, _ in testloader:\n        if cuda:\n            x_test = x_test.cuda()\n        for k in range(x_test.shape[0]):\n            l+=1\n            x_testFID = x_test[k]\n            torchvision.utils.save_image(x_testFID, os.path.join('/content/drive/MyDrive/samples/', f'{l}.png'))"

#3 - Training the diffusion model:

In [15]:
num_iterations=10000
@torch.no_grad()
def evaluate(gen_sde):
    test_bpd = list()
    gen_sde.eval()
    for x_test, _ in testloader:
        if cuda:
            x_test = x_test.cuda()
        x_test = x_test * 255 / 256 + torch.rand_like(x_test) / 256
        if real:
            x_test, ldj = logit.forward_transform(x_test, 0)
            elbo_test = gen_sde.elbo_random_t_slice(x_test)
            elbo_test += ldj
        else:
            elbo_test = gen_sde.elbo_random_t_slice(x_test)
        test_bpd.extend(- (elbo_test.data.cpu().numpy() / dimx) / np.log(2) + 8)
         
    
    gen_sde.train()
    test_bpd = np.array(test_bpd)
    return test_bpd.mean(), test_bpd.std() / len(testloader.dataset.data) ** 0.5

@torch.no_grad()
def generate(gen_sde):
    print("Beginning of Generation ....")
    for j in range(1024):    
        with torch.no_grad():
                  print("Begin Generation of image ....",j)
                  mean=0
                  std=1
                  delta = gen_sde.T / 50
                  y0 = torch.randn(1, input_channels, input_height, input_height).to(gen_sde.T)
                  y0 = y0 * std + mean
                  ts = torch.linspace(0, 1, 50 + 1).to(y0) * gen_sde.T
                  ones = torch.ones(1, 1, 1, 1).to(y0)
                  for i in range(50):
                        mu = gen_sde.mu(ones * ts[i], y0)
                        sigma = gen_sde.sigma(ones * ts[i], y0)
                        y0 = y0 + delta * mu + delta ** 0.5 * sigma * torch.randn_like(y0)
                  y0 = torch.clip(y0, 0, 1)
                  y0 = y0.view(1, 1, input_channels, input_height, input_height).permute(2, 0, 3, 1, 4).contiguous().view(input_channels, 1 * input_height, 1 * input_height)
                  
                  torchvision.utils.save_image(y0, os.path.join('/content/drive/MyDrive/generated/', f'{j}.png'))
                  print("Generation of image ....",j)
    print("End of Generation ....")
     

if os.path.exists(os.path.join(folder_path, 'checkpoint.pt')):
    gen_sde, optim, not_finished, count = torch.load(os.path.join(folder_path, 'checkpoint.pt'))
else:

    not_finished = True

    count = 0

    writer.add_scalar('T', sde_model.T.item(), count)

    writer.add_image('samples',
                     get_grid(sde_model, input_channels, input_height, n=4,
                              num_steps= num_steps, transform=reverse),
                     0)    
while not_finished:

    for x, _ in trainloader:

        if cuda:
            x = x.cuda()
        x = x * 255 / 256 + torch.rand_like(x) / 256
        if real:
            x, _ = logit.forward_transform(x, 0)

        loss = sde_model.dsm(x).mean()

        optim.zero_grad()
        loss.backward()
        optim.step()

        count += 1
        print("step .....",count)
        if count == 1 or count % print_every == 0:
            writer.add_scalar('loss', loss.item(), count)
            writer.add_scalar('T', sde_model.T.item(), count)
            bpd, std_err = evaluate(sde_model)
            writer.add_scalar('bpd', bpd, count)
            print_(f'Iteration {count} \tBPD {bpd}')

        if count >= num_iterations:
            not_finished = False
            print_('Finished training')
            break

        if count % sample_every == 0:
            sde_model.eval()
            writer.add_image('samples',
                             get_grid(sde_model, input_channels, input_height, n=4,
                                      num_steps=num_steps, transform=reverse),
                             count)
            sde_model.train()
            print("sampling",count)
        if count % checkpoint_every == 0:
            torch.save([sde_model, optim, not_finished, count], os.path.join(folder_path, 'checkpoint.pt'))
            print("checkpoint",count)
torch.save(sde_model, os.path.join('/content/drive/MyDrive/Colab Notebooks/modified version/Checkpoint', 'bestmodel.pth'))

Streaming output truncated to the last 5000 lines.
step ..... 5025
step ..... 5026
step ..... 5027
step ..... 5028
step ..... 5029
step ..... 5030
step ..... 5031
step ..... 5032
step ..... 5033
step ..... 5034
step ..... 5035
step ..... 5036
step ..... 5037
step ..... 5038
step ..... 5039
step ..... 5040
step ..... 5041
step ..... 5042
step ..... 5043
step ..... 5044
step ..... 5045
step ..... 5046
step ..... 5047
step ..... 5048
step ..... 5049
step ..... 5050
step ..... 5051
step ..... 5052
step ..... 5053
step ..... 5054
step ..... 5055
step ..... 5056
step ..... 5057
step ..... 5058
step ..... 5059
step ..... 5060
step ..... 5061
step ..... 5062
step ..... 5063
step ..... 5064
step ..... 5065
step ..... 5066
step ..... 5067
step ..... 5068
step ..... 5069
step ..... 5070
step ..... 5071
step ..... 5072
step ..... 5073
step ..... 5074
step ..... 5075
step ..... 5076
step ..... 5077
step ..... 5078
step ..... 5079
step ..... 5080
step ..... 5081
step ..... 5082
step ..... 5083
step 